In [ ]:
import sys

sys.path.insert(1, "../")

import rasterio as rio
import numpy as np
from shapely import from_wkt
from dem_handler.utils.spatial import transform_polygon
from dem_comparison.analysis import analyse_difference_for_interval
from dem_comparison.utils import simple_mosaic, kml_to_poly
from dem_comparison.utils import get_cross_section_data, enhance_image, hillshade
from dem_comparison.plots import plot_cross_sections, plot_slope_vs_height
from pathlib import Path
import os
import glob
from dem_comparison.utils import reproject_match_tifs
from dem_handler.utils.rio_tools import (
    reproject_profile_to_new_crs,
    reproject_arr_to_match_profile,
    reproject_arr_to_new_crs,
)
import shutil
import matplotlib.pyplot as plt
import gemgis as gg
from shapely import Polygon
from dem_comparison.analysis import area_of_interest
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import aioboto3
import yaml
from dem_comparison.utils import bulk_upload_files

In [ ]:
other_aoi_names = [
    "Amery_1",
    "Amery_2",
    "Queen_Mnt_1",
    "Queen_Mnt_2",
    "Artifact_2",
    "Artifact_3",
    "Sastrugi",
    "Amery_Blue_Ice",
    "Jelbart",
]
other_aoi_polys = list(kml_to_poly("../temp/cross_section_aois.kml").geoms)
# aoi_name = "Artifact_1"
# aoi_name = "Ellsworth_Mountains_Lower"
# aoi_name = "Sør_AOI"
aoi_name = other_aoi_names[8]
Path(f"../temp/{aoi_name}_Outputs/Plots").mkdir(parents=True, exist_ok=True)

poly_crs = 4326  # 3031

In [ ]:
# poly = from_wkt("POLYGON ((-1478972.5805124296 -584351.9503323506, -1429491.2228967457 -609735.2441741625, -1466227.382338693 -677424.0277523277, -1514959.0224147453 -653861.4765067638, -1478972.5805124296 -584351.9503323506))")
# poly = kml_to_poly(f"{aoi_name}_Outputs/aoi.kml")
poly = other_aoi_polys[other_aoi_names.index(aoi_name)]
poly.bounds

In [ ]:
output_files = area_of_interest(
    poly,
    poly_crs,
    aoi_name,
    # return_outputs=True,
    keep_temp_files=True,
    # s3_upload_dir="auto",
    save_path=f"../temp/{aoi_name}_Outputs",
    temp_path=Path(f"../temp/TEMP_{aoi_name}"),
)

In [ ]:
major_axis_ratio = 0.2
minor_axis_ratio = 0.55
res_multipllier = 1.1
h_factor = 0.5
w_factor = 0.7

with rio.open(output_files[0]) as ds:
    w = ds.width
    h = ds.height

aspect = w / h
print(h, w, aspect)
plot_res = (
    int(h_factor * res_multipllier * h),
    int(w_factor * res_multipllier * h * aspect),
)
plot_res

In [ ]:
hcs_fig = plot_cross_sections(
    output_files[1:3],
    transform_polygon(poly, poly_crs, 3031),
    output_files[0],
    raster_names=["REMA", "COP"],
    raster_colours=["orange", "cyan"],
    save_path=Path(f"../temp/{aoi_name}_Outputs/Plots/height_cross_plot.html"),
    full_map=Path("../resources/mosaic_downsampled_3200m.tif"),
    aoi_name=aoi_name.replace("_", " "),
    major_axis_ratio=major_axis_ratio,
    minor_axis_ratio=minor_axis_ratio,
    diff_opacity=0.0,
    hillshade_index=1,
    aoi_buffer=10,
    dynamic_spacing=False,
    plot_resolution=plot_res,
    # preview=True
)

In [ ]:
hcs_fig

In [ ]:
scs_fig = plot_cross_sections(
    output_files[4:6],
    transform_polygon(poly, poly_crs, 3031),
    output_files[3],
    raster_names=["REMA", "COP"],
    raster_colours=["orange", "cyan"],
    save_path=f"../temp/{aoi_name}_Outputs/Plots/slope_cross_plot.html",
    axes_label="Slope(°)",
    full_map=Path("../resources/mosaic_downsampled_3200m.tif"),
    aoi_name=aoi_name.replace("_", " "),
    major_axis_ratio=major_axis_ratio,
    minor_axis_ratio=minor_axis_ratio,
    diff_unit="°",
    diff_opacity=0.0,
    hillshade_index=1,
    aoi_buffer=10,
    dynamic_spacing=False,
    # plot_resolution=plot_res,
)

In [ ]:
scs_fig

In [ ]:
hvss_fig = plot_slope_vs_height(
    output_files[5:3:-1],
    output_files[0],
    output_files[3],
    transform_polygon(poly, poly_crs, 3031),
    raster_names=["REMA", "COP"],
    save_path=Path(f"../temp/{aoi_name}_Outputs/Plots/height_slope_plot.html"),
    aoi_name=aoi_name.replace("_", " "),
    major_axis_ratio=major_axis_ratio,
    minor_axis_ratio=minor_axis_ratio,
    plot_trendline=True,
)

In [ ]:
hvss_fig

In [ ]:
with open("../temp/credentials_aws.yaml") as f:
    cred_dict = yaml.safe_load(f)
AWS_ACCESS_KEY_ID = cred_dict["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = cred_dict["AWS_SECRET_ACCESS_KEY"]
AWS_DEFAULT_REGION = cred_dict["AWS_DEFAULT_REGION"]

session = aioboto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION,
)
s3_dir = Path(f"dem_comparison_results/Areas_of_Interest/{aoi_name}_Outputs")
local_dir = Path(f"../temp/{aoi_name}_Outputs")
bulk_upload_files(
    s3_dir,
    local_dir,
    session=session,
    num_cpus=4,
)